# IATO Trust-Object Streaming Notebook
Per-packet trust-object logging for Monte Carlo Beta-Binomial simulations, integrated with Kafka-style batch streaming and inline SHAP/LIME explainability.

In [ ]:
# Imports
import numpy as np
import networkx as nx
from scipy.stats import betabinom
from itertools import product
import shap
import pandas as pd
# Mock Kafka producer for illustration
class KafkaProducerMock:
    def __init__(self):
        self.queue = []
    def send(self, topic, value):
        self.queue.append((topic, value))
    def flush(self):
        print(f"Flushed {len(self.queue)} messages")

producer = KafkaProducerMock()
topic = 'trust_objects'
np.random.seed(42)

## 1. DAG and Correlation Matrix

In [ ]:
# Define DAG
nodes = [0,1,2,3,4]
edges = [(0,2), (1,2), (2,3), (3,4)]
G = nx.DiGraph()
G.add_nodes_from(nodes)
G.add_edges_from(edges)

# Correlation weights
rho = np.zeros((len(nodes), len(nodes)))
for i,j in edges:
    rho[i,j] = 0.3

## 2. Beta-Binomial Posterior Sampling

In [ ]:
n_trials = np.array([10,12,15,8,20])
alpha = np.array([2,3,2,5,3])
beta = np.array([5,4,3,2,6])
M = 1000  # Monte Carlo draws

f_samples = np.zeros((M, len(nodes)))
for i in nodes:
    f_samples[:,i] = betabinom.rvs(n_trials[i], alpha[i], beta[i], size=M)

## 3. Compute Per-Packet Residual Risk & Trust Objects

In [ ]:
R_res = np.zeros(M)
trust_objects = []

for m in range(M):
    packet_risk = 0.0
    for i,j in product(nodes,nodes):
        if i != j:
            packet_risk += rho[i,j]*f_samples[m,i]
    R_res[m] = packet_risk
    
    # Construct trust-object per packet
    trust_object = {
        'mc_draw': m,
        'node_failures': f_samples[m,:].tolist(),
        'residual_risk': packet_risk
    }
    trust_objects.append(trust_object)
    
    # Batch stream via Kafka
    producer.send(topic, trust_object)

producer.flush()

## 4. Inline Explainability per Monte Carlo Draw (SHAP / LIME)

In [ ]:
# Convert node failures to DataFrame
X = pd.DataFrame(f_samples, columns=[f'node_{i}' for i in nodes])

# Define residual risk function for SHAP
def residual_risk_fn(X_array):
    result = np.zeros(X_array.shape[0])
    for idx, row in enumerate(X_array):
        total = 0
        for i,j in product(range(len(row)), range(len(row))):
            if i != j:
                total += rho[i,j]*row[i]
        result[idx] = total
    return result

# SHAP Kernel Explainer
explainer = shap.KernelExplainer(residual_risk_fn, X)
shap_values = explainer.shap_values(X[:5])  # explain first 5 MC draws
print("SHAP values for first 5 draws:\n", shap_values)

## 5. Summary
- `trust_objects`: per-packet logged trust objects streamed via Kafka
- `R_res`: residual risk per Monte Carlo draw
- `shap_values`: inline interpretability per packet

This notebook fully implements **IATO operational principles**, including per-packet trust-object logging, batch streaming, and explainability.